In [ ]:
import copy

import pandas as pd

from constraint_relax import search_mfs
from experiment_constraint_data_processor import process_experiment_constraint_data
from values import real_data_path, simulated_data_path
from similarity_calculator import compute_similarity
from utils import get_df_len

In [ ]:
problem_list = process_experiment_constraint_data('./experiment_constraint_data.csv')

In [ ]:
sol_list = []
result = pd.DataFrame()
threshold_list = ['', 0, 0.5, 1]
for problem in problem_list:
    prob_sol_list = dict()
    for threshold in threshold_list:
        p = copy.deepcopy(problem)
        if threshold != '':
            mfs = search_mfs(p.constraint_list, copy.copy(p.constraint_list), simulated_data_path)
            p.relax(mfs, threshold)
        prob_items = p.solve()
        prob_sol_list.update({
            f'constraint_{threshold}': p.constraint_list,
            f'no_of_items_{threshold}': get_df_len(prob_items)
        })
        if not prob_items is None:
            prob_sol_list.update({
                f'similarity_{threshold}': ((compute_similarity(p.constraint_list, prob_items))['similarity']).mean()
            })
        else:
            prob_sol_list.update({
                f'similarity_{threshold}': 0
            })
    result = result.append(prob_sol_list, ignore_index=True)

In [ ]:
result

In [ ]:
pd.value_counts(result.iloc[:, 1] == 0)

In [ ]:
valid = ((result.iloc[:, 1] <= result.iloc[:, 4]) & (result.iloc[:, 4] <= result.iloc[:, 7]) & (result.iloc[:, 7] <= result.iloc[:, 10]))
valid_count = pd.value_counts(valid)
print(valid_count)

In [ ]:
valid = ((result.iloc[:, 1] < result.iloc[:, 4]) & (result.iloc[:, 4] < result.iloc[:, 7]) & (result.iloc[:, 7] < result.iloc[:, 10]))
valid_count = pd.value_counts(valid)
print(valid_count)

In [ ]:
result[valid]

In [6]:
valid = ((result.iloc[:, 1] <= result.iloc[:, 4]) & (result.iloc[:, 4] <= result.iloc[:, 7]) & (result.iloc[:, 7] <= result.iloc[:, 10]))
valid_count = pd.value_counts(valid)
print(valid_count)

True     471
False     29
dtype: int64


In [7]:
valid = ((result.iloc[:, 1] < result.iloc[:, 4]) & (result.iloc[:, 4] < result.iloc[:, 7]) & (result.iloc[:, 7] < result.iloc[:, 10]))
valid_count = pd.value_counts(valid)
print(valid_count)

False    498
True       2
dtype: int64


In [8]:
result[valid]

,constraint_,no_of_items_,similarity_,constraint_0,no_of_items_0,similarity_0,constraint_0.5,no_of_items_0.5,similarity_0.5,constraint_1,no_of_items_1,similarity_1
229,"{weight: 2.47: 4: Nature.LESS, price: 4706.33:...",12,0.661046,"{weight: 2.47: 4: Nature.LESS, os: 10: 6: Natu...",14,0.658962,"{price: : 1: Nature.LESS, brand: lenovo|huawei...",16,0.633422,"{weight: 2.47: 4: Nature.LESS, os: 10: 6: Natu...",47,0.619379
482,"{weight: 1.52: 2: Nature.LESS, ram: 8.0: 4: Na...",0,0.000000,"{ram: 8.0: 4: Nature.MORE, storage: 512: 5: Na...",6,0.674799,"{storage: 512: 5: Nature.MORE, price: 5056.09:...",8,0.595528,"{ram: 8.0: 4: Nature.MORE, storage: : 5: Natur...",9,0.527914
